In [ ]:
print('Python')

In [ ]:
# TensorFlow 설치
%pip install tensorflow

In [ ]:
import tensorflow as tf; print(tf.reduce_sum(tf.random.normal([1000, 1000])))

In [ ]:
%pip install tensorflow-datasets

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import tensorflow as tf
import numpy as np
x = np.random.sample((100, 3))      # 행, 열 형태로 난수 생성
dataset = tf.data.Dataset.from_tensor_slices(x)
dataset

In [ ]:
# tensorflow_datasets 패키지 호출
import tensorflow_datasets as tfds
ds = tfds.load('mnist', split='train', shuffle_files=True)

In [10]:
# 케라스에서 제공하는 데이터셋 사용
import tensorflow as tf
data_train, data_test = tf.keras.datasets.mnist.load_data()     # 케라스를 이용한 MNIST 데이터셋 내려받기
(image_train, labels_train) = data_train
(image_test, labels_test) = data_test

In [ ]:
# 인터넷에서 데이터셋을 로컬 컴퓨터에 내려받아 사용
# tf.keras.utils.get_file 메소드 사용
import tensorflow as tf
url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt'
text_path = tf.keras.utils.get_file("butler.txt", origin=url)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")

cols = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety', 'output']
# pandas는 dataframe으로 csv 파일을 읽어올 수 있는 함수를 가지고 있음
# pd.read_csv() 함수를 통해 데이터를 데이터 프레임으로 불러옴
cars = pd.read_csv('data/car_evaluation.csv', names=cols, header=None)
cars.info()

plot_size = plt.rcParams["figure.figsize"]
plot_size [0] = 8
plot_size [1] = 6
plt.rcParams["figure.figsize"] = plot_size
cars.output.value_counts().plot(
    kind='pie', autopct='%0.05f%%', 
    colors=['lightblue', 'lightgreen', 'orange', 'pink'], 
    explode=(0.05,0.05,0.05,0.05)
    )


In [ ]:
# 전처리 작업 - 범위 정보를 변환하기 위한 작업
# 원-핫 인코딩
price = pd.get_dummies(cars.price, prefix='price')
maint = pd.get_dummies(cars.maint, prefix='maint')
doors = pd.get_dummies(cars.maint, prefix='doors')
persons = pd.get_dummies(cars.persons, prefix='persons')
lug_capacity = pd.get_dummies(cars.lug_capacity, prefix='lug_capacity')
safety= pd.get_dummies(cars.safety, prefix='safety')
labels= pd.get_dummies(cars.output, prefix='conditions')

X = pd.concat([price, maint, doors, persons, lug_capacity, safety], axis=1)
Y = labels.values       # numpy의  array로 변환       *R에서 사용하는 배열과 같다고 생각하면 됨

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.20, random_state=42
)

# 모델 생성 및 컴파일
from tensorflow.keras.layers import Input, Dense, Activation, Dropout # type: ignore
from tensorflow.keras.models import Model # type: ignore

input_layer = Input(shape=(X.shape[1], ))
dense_layer_1 = Dense(15, activation='relu')(input_layer)
dense_layer_2 = Dense(10, activation='relu')(dense_layer_1)
output = Dense(Y.shape[1], activation='softmax')(dense_layer_2)

model = Model(inputs=input_layer, outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

In [ ]:
# 모델 훈련
model.fit(X_train, Y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2)

In [ ]:
# 모델 평가
score = model.evaluate(X_test, Y_test, verbose=1)

print("Test Score : ", score[0])            
print("Test Accuracy : ", score[1])

In [ ]:
# 텐서 보드 사용 - 훈련 과정 모니터링
# 1. log_dir 지정 (./log/fit/)
log_dir = "logs/fit/"
# 2. tensorboard_callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1
)
model.fit(X_train, Y_train, batch_size=8, epochs=50, verbose=1, validation_split=0.2,
          callbacks=[tensorboard_callback])

In [ ]:
# tensorboard 서버 실행
!tensorboard --logdir=./logs/fit/